In [3]:
import os
from langchain import PromptTemplate
import openai
import pinecone
from fastapi.middleware.cors import CORSMiddleware
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import QA_PROMPT
from dotenv import load_dotenv, find_dotenv
from langchain.llms import AzureOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA

#check .env
_ = load_dotenv(find_dotenv())

# Open AI Chat Completion LLM Model from Langchain, with Streaming enabled
openai_llm = AzureOpenAI(
    deployment_name="gpt-tech-snacks",
    model_name="text-embedding-ada-002",
    temperature=0
)

embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'],
                              #this makes it work with azure
                              deployment="embedding-tech-snacks",
                              model="text-embedding-ada-002",
                              openai_api_base=os.environ['OPENAI_API_BASE'],
                              openai_api_type=os.environ['OPENAI_API_TYPE']
                              )#1 is azure limitation
TEXT_FIELD = "text"
index = pinecone.Index(os.environ['PINECONE_INDEX'])
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [6]:
namespace="helicopter"
#namespace="sphinx"

In [13]:
vectorstore = Pinecone(
    index, embeddings.embed_query, TEXT_FIELD,
    namespace=namespace
)
qa = RetrievalQA.from_chain_type(
    llm=openai_llm,
    chain_type="stuff",    
    retriever=vectorstore.as_retriever(),
    memory=memory
)


In [15]:
response = qa.run("How to take-off?")
print(response)

 
1. Complete the pre-take-off checklist and ensure the area is clear. 
2. Use the cyclic to maintain position over the ground during the takeoff. Before starting the takeoff, the cyclic is centralized, and then adjusted as required to ascend vertically and prevent the helicopter from drifting. 
3. Use the collective to lift off and climb from the ground to the normal hover height of 3 to 5 feet. The collective should be raised slowly so the other controls can be adjusted, as the helicopter becomes “light on the skids”; this will ensure that an accurate takeoff is accomplished. 
4. Use the pedals to control the heading of the helicopter. Prior to take off the pedals are centralized. As torque is increased pedal will have to be applied to maintain heading. 
5. Throttle may have to be adjusted in some piston engine helicopter to maintain RPM throughout the takeoff. 
6. After every takeoff, a hover check must be completed. 
7. If possible make your takeoffs and landings into wind.<|im_end